题目1：根据对BN,LN,IN,GN的理解，完成BLOCK 1.1,1.2两处程序填空并正确运行
题目2：完成去掉normalization的程序版本并比较实验结果，同时自行查阅资料分析normalization的作用
题目3：查阅资料了解CIFAR10，CIFAR100数据集上resnet表现，改进该网络中不适用于CIFAR数据集的结构
//斜线内删掉
其中一些可改进的问题：
1.第一层卷积kernel_size过大，不适用于CIFAR10/100小尺寸图片
2.同理，第一层卷积后maxpool过早
//
附加题bonus：基本复现resnet在CIFAR上的最好评分

In [1]:
import os
import torch
import yaml
from torchvision import datasets
import torch.nn as nn
import torchvision.models as models
import numpy as np
import time
print(torch.__version__)
torch.manual_seed(0)

1.6.0


In [2]:
args = {
    "data": {
        "dataroot":"/mnt/pami14/zengyi/dataset/CIFAR100/",
        "dataset":"CIFAR100",
        "class_num":10,
        "train": {
            "numWorkers": 2,
            "batchSize": 128,
            "shuffle": True,
        },
        "test": {
            "numWorkers": 2,
            "batchSize": 128,
            "shuffle": False
        }
    },
    "model": {
        "norm_type":"GN",
        "group_num":4,
        "epochs": 200,
        "testFreq": 2,
        "optimizer": "Adam",
        "learningRate": 1e-3,
        "weightDecay": 0,
        "modelsave":True,
        "modelload":False,
        "dropout":0.0,
        "isDropout":True
    },
    "randomseed": 1007,
    "CUDA_VISIBLE_DEVICES": "1",
    "use_gpu": True
}
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# CIFAR-10 Dataset

In [3]:
from torchvision.datasets import CIFAR10,CIFAR100
from torchvision import transforms

In [4]:
trainFolder=None
testFolder=None
trans=transforms.Compose([
    transforms.ToTensor(),
])

if(args["data"]["dataset"]=="CIFAR10"):
    args["data"]["class_num"]=10
    trainFolder=CIFAR10(args["data"]["dataroot"],True,transform=trans,download=True)
    testFolder=CIFAR10(args["data"]["dataroot"],False,transform=trans,download=True)
elif(args["data"]["dataset"]=="CIFAR100"):
    args["data"]["class_num"]=100
    trainFolder=CIFAR10(args["data"]["dataroot"],True,transform=trans,download=True)
    testFolder=CIFAR10(args["data"]["dataroot"],False,transform=trans,download=True)
else:
    raise ValueError('dataset name unknown:',args["data"]["dataset"])
#(data,target)=trainFolder.__getitem__(0)
#print(type(data),type(target))
#print(data.size())

Extracting /mnt/pami14/zengyi/dataset/CIFAR100/cifar-10-python.tar.gz to /mnt/pami14/zengyi/dataset/CIFAR100/
Files already downloaded and verified


In [5]:
trainLoader = torch.utils.data.DataLoader(
    trainFolder,
    batch_size=args['data']['train']['batchSize'] * torch.cuda.device_count(),
    shuffle=args['data']['train']['shuffle'],
    num_workers=args['data']['train']['numWorkers'] * torch.cuda.device_count(),
    drop_last=True,
    pin_memory=True)
testLoader = torch.utils.data.DataLoader(
    testFolder,
    batch_size=args['data']['test']['batchSize'] * torch.cuda.device_count(),
    shuffle=args['data']['test']['shuffle'],
    num_workers=args['data']['test']['numWorkers'] * torch.cuda.device_count(),
    drop_last=True,
    pin_memory=True)

#for i, (img, label) in enumerate(trainLoader):
#    print(img.size(),label.size())
#    break

# Resnet Backbone

In [6]:
import os
import torch
from torch import Tensor
from torchvision import datasets
import torch.nn as nn
from torchvision.models import ResNet
from torchvision.models.resnet import load_state_dict_from_url,model_urls,conv1x1,conv3x3
from typing import Type, Any, Callable, Union, List, Optional

In [7]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, norm_type="BN", group_num=4, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            if(norm_type=="BN"):
                norm_layer = nn.BatchNorm2d
            else:
                norm_layer = nn.GroupNorm

        self.norm_type=norm_type
        self.group_num=group_num
        
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        #self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        #self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride
        
        if(self.norm_type=="BN"):
            self.bn1 = norm_layer(planes * block.expansion)
            self.bn2 = norm_layer(planes * block.expansion)
        elif(self.norm_type=="LN"):
            self.bn1 = norm_layer(1,planes * block.expansion)
            self.bn2 = norm_layer(1,planes * block.expansion)
        elif(self.norm_type=="IN"):
            self.bn1 = norm_layer(planes * block.expansion,planes * block.expansion)
            self.bn2 = norm_layer(planes * block.expansion,planes * block.expansion)
        elif(self.norm_type=="GN"):
            self.bn1 = norm_layer(self.group_num,planes * block.expansion)
            self.bn2 = norm_layer(self.group_num,planes * block.expansion)
        else:
            raise ValueError('norm type not in BN,LN,IN,GN:',self.norm_type)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    # Bottleneck in torchvision places the stride for downsampling at 3x3 convolution(self.conv2)
    # while original implementation places the stride at the first 1x1 convolution(self.conv1)
    # according to "Deep residual learning for image recognition"https://arxiv.org/abs/1512.03385.
    # This variant is also known as ResNet V1.5 and improves accuracy according to
    # https://ngc.nvidia.com/catalog/model-scripts/nvidia:resnet_50_v1_5_for_pytorch.

    expansion = 4

    def __init__(self, inplanes, planes, stride=1, norm_type="BN", group_num=4, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            if(norm_type=="BN"):
                norm_layer = nn.BatchNorm2d
            else:
                norm_layer = nn.GroupNorm

        self.norm_type=norm_type
        self.group_num=group_num
        
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        #self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        #self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        #self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        
        #####BLOCK 1.1#####
        #程序填空BLOCK 1.1
        #LN,IN,GN参数填空
        #提交时删掉该注释部分
        '''
        if(self.norm_type=="BN"):
            self.bn1 = norm_layer(width)
            self.bn2 = norm_layer(width)
            self.bn3 = norm_layer(planes * self.expansion)
        elif(self.norm_type=="LN"):
            self.bn1 = norm_layer(1,width)
            self.bn2 = norm_layer(1,width)
            self.bn3 = norm_layer(1,planes * self.expansion)
        elif(self.norm_type=="IN"):
            self.bn1 = norm_layer(width,width)
            self.bn2 = norm_layer(width,width)
            self.bn3 = norm_layer(planes * self.expansion,planes * self.expansion)
        elif(self.norm_type=="GN"):
            self.bn1 = norm_layer(self.group_num,width)
            self.bn2 = norm_layer(self.group_num,width)
            self.bn3 = norm_layer(self.group_num,planes * self.expansion)
        '''
        #填空题1.1:仿造BN格式对LN,IN,GN填空
        
        if(self.norm_type=="BN"):
            self.bn1 = norm_layer(width)
            self.bn2 = norm_layer(width)
            self.bn3 = norm_layer(planes * self.expansion)
        elif(self.norm_type=="LN"):
            #LN代码块
        elif(self.norm_type=="IN"):
            #IN代码块
        elif(self.norm_type=="GN"):
            #GN代码块
        
        else:
            raise ValueError('norm type not in BN,LN,IN,GN:',self.norm_type)

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class resnet_backbone(ResNet):
    
    def __init__(self, block, layers, norm_type="BN", group_num=4, num_classes=1000, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            if(norm_type=="BN"):
                norm_layer = nn.BatchNorm2d
            else:
                norm_layer = nn.GroupNorm
        self._norm_layer = norm_layer

        self.norm_type=norm_type
        self.group_num=group_num
        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)
                    
    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            
            #####BLOCK 1.2#####
            #程序填空BLOCK 1.2.
            #LN,IN,GN参数填空
            #提交时删掉该注释部分
            '''
            if(self.norm_type=="BN"):
                #print("isBN")
                downsample = nn.Sequential(
                    conv1x1(self.inplanes, planes * block.expansion, stride),
                    norm_layer(planes * block.expansion),
                )
            elif(self.norm_type=="LN"):
                #print("isLN")
                downsample = nn.Sequential(
                    conv1x1(self.inplanes, planes * block.expansion, stride),
                    norm_layer(1,planes * block.expansion),
                )
            elif(self.norm_type=="IN"):
                #print("isIN")
                downsample = nn.Sequential(
                    conv1x1(self.inplanes, planes * block.expansion, stride),
                    norm_layer(planes * block.expansion,planes * block.expansion),
                )
            elif(self.norm_type=="GN"):
                #print("isGN")
                downsample = nn.Sequential(
                    conv1x1(self.inplanes, planes * block.expansion, stride),
                    norm_layer(self.group_num,planes * block.expansion),
                )
            '''
            
            #填空题1.2:仿造BN格式对LN,IN,GN填空
            if(self.norm_type=="BN"):
                #####
            elif(self.norm_type=="LN"):
                #####
            elif(self.norm_type=="IN"):
                #####
            elif(self.norm_type=="GN"):
                #####
            
            else:
                raise ValueError('norm type not in BN,LN,IN,GN:',self.norm_type)
            #print(planes,block.expansion)

        layers = []
        layers.append(block(self.inplanes, planes, stride, self.norm_type, self.group_num, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, norm_type=self.norm_type, group_num=self.group_num, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)
    
    def _forward_impl(self, x: Tensor) -> Tensor:
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        #print(x.size())
        x = self.layer1(x)
        #print(x.size())
        x = self.layer2(x)
        #print(x.size())
        x = self.layer3(x)
        #print(x.size())
        x = self.layer4(x)
        #print(x.size())

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        x = torch.nn.functional.sigmoid(x)
        return x

    def forward(self, x: Tensor) -> Tensor:
        return self._forward_impl(x)
    
def _resnet(
    arch: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    norm_type: str,
    group_num: int,
    **kwargs: Any
) -> ResNet:
    model = resnet_backbone(block, layers, norm_type, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained: bool = False, progress: bool = True, norm_type: str = "BN", group_num: int = 4, **kwargs: Any) -> ResNet:
    r"""ResNet-18 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet18', Bottleneck, [2, 2, 2, 2], pretrained, progress, norm_type, group_num,
                   **kwargs)


def resnet34(pretrained: bool = False, progress: bool = True, norm_type: str = "BN", group_num: int = 4, **kwargs: Any) -> ResNet:
    r"""ResNet-34 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet34', Bottleneck, [3, 4, 6, 3], pretrained, progress, norm_type, group_num,
                   **kwargs)


def resnet50(pretrained: bool = False, progress: bool = True, norm_type: str = "BN", group_num: int = 4, **kwargs: Any) -> ResNet:
    r"""ResNet-50 model from
    `"Deep Residual Learning for Image Recognition" <https://arxiv.org/pdf/1512.03385.pdf>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet('resnet50', Bottleneck, [3, 4, 6, 3], pretrained, progress, norm_type, group_num,
                   **kwargs)

# Initialization

In [8]:
seed = args['randomseed']
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
model=resnet18(norm_type=args["model"]["norm_type"], group_num=args["model"]["group_num"], num_classes=args["data"]["class_num"])
epoch=0

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=args['model']['learningRate'],
    weight_decay=args['model']['weightDecay']
)



if(args["use_gpu"]):
    model = torch.nn.DataParallel(model).cuda()

# Train

## Loss & Record

In [9]:
def loss_function_CE(p, y):
    from torch import log
    #p=[N,C,T,W,H]
    delta = 1e-4
    p = -y*log(p+delta)-(1-y) *log(1-p+delta)
    return p.sum()

In [10]:
class Record(object):
    def __init__(self):
        self.loss = 0
        self.count = 0

        self.miniloss = 0
        self.minicount = 0

    def add(self, value):
        self.loss += value
        self.count += 1
        self.minicount += 1
        self.miniloss += value
        
    def add_2(self, value1, value2):
        self.loss+=value1
        self.count+=value2

    def mean(self):
        return self.loss / self.count

    def minimean(self):
        ans = self.miniloss / self.minicount
        self.minicount = 0
        self.miniloss = 0
        return ans

## Train

In [11]:
def train():
    #print("start training...")
    model.train()
    record = Record()
    current_time = time.time()
    #print(current_time)
    for i, (img, label) in enumerate(trainLoader):
        img = img.float()
        label = label
        label = torch.unsqueeze(label,dim=1)
        
        if(args["use_gpu"]):
            img=img.cuda()
            label=label.cuda()
            label=torch.cuda.FloatTensor(label.size(0), args["data"]["class_num"]).fill_(0).scatter_(1, label, 1)
        else:
            label=torch.zeros(label.size(0), args["data"]["class_num"]).scatter_(1, label, 1)
        
        #print(img.size(),label.size())
        #print(img.type(),label.type())
        
        y1= model(img)
        #print(y1.type(),label.type())
        loss = loss_function_CE(y1,label)/y1.size(0)
        #print(y1.size(0),loss.data)
        optimizer.zero_grad()
        record.add(loss.data)
        loss.backward()
        optimizer.step()
        #print(trainLoader.__len__(),testLoader.__len__())
        print("\r"+f'The epoch :{epoch},batch:{(i*100/trainLoader.__len__()):.2f}%,The training loss is:{loss.data:.4f}',end="")
            
    #print(' ')
    #print(
    #    f'----------Iteration per epoch :{len(trainLoader)}--------------------')
    #print(f'The epoch :{epoch}')
    #print(f'The epoch cost time:{(time.time()-current_time):.2f}s')
    #print(f'The training loss is:{record.mean():.4f}')
    return record.mean(), (time.time()-current_time)

## Test

In [12]:
def test():
    #print("start training...")
    model.eval()
    record = Record()
    current_time = time.time()
    #print(current_time)
    with torch.no_grad():
        for i, (img, label) in enumerate(testLoader):
            img = img.float()
            label = label
            label = torch.unsqueeze(label,dim=1)

            if(args["use_gpu"]):
                img=img.cuda()
                label=label.cuda()
                label=torch.cuda.FloatTensor(label.size(0), args["data"]["class_num"]).fill_(0).scatter_(1, label, 1)
            else:
                label=torch.zeros(label.size(0), args["data"]["class_num"]).scatter_(1, label, 1)

            #print(img.size(),label.size())
            #print(img.type(),label.type())

            y1= model(img)
            #print(y1.type(),label.type())
            loss = loss_function_CE(y1,label)/y1.size(0)
            y1=(y1>=0.5)*label
            
            ct=torch.sum(y1).data
            total=y1.size(0)
            
            #print(y1.size(0),loss.data)
            #optimizer.zero_grad()
            record.add_2(ct,total)
            #loss.backward()
            #optimizer.step()
            #print(trainLoader.__len__(),testLoader.__len__())
            print("\r"+f'The epoch :{epoch},batch:{(i*100/trainLoader.__len__()):.2f}%,The testing accuracy is:{ct/total:.4f}',end="")
            
    print(' ')
    print(
        f'----------Iteration per epoch :{len(trainLoader)}--------------------')
    print(f'The epoch :{epoch}')
    print(f'The epoch cost time:{(time.time()-current_time):.2f}s')
    print(f'The accuracy is:{record.mean():.4f}')
    return record.mean()

In [13]:
#Encoder = resnet50(pretrained=False,norm_type="GN")
#testinput=torch.ones(100,3,64,64)
#testinput=Encoder(testinput)
#print(testinput.size())

# Run

In [ ]:
for e in range(args["model"]["epochs"]):
    train()
    if(epoch%args["epoch_report_inter"]==0):
        test()
    epoch+=1

/home/zengyi/anaconda3/lib/python3.8/site-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


The epoch :0,batch:19.74%,The testing accuracy is:0.2422 
----------Iteration per epoch :390--------------------
The epoch :0
The epoch cost time:1.08s
The accuracy is:0.2404
The epoch :10,batch:19.74%,The testing accuracy is:0.6406 
----------Iteration per epoch :390--------------------
The epoch :10
The epoch cost time:1.07s
The accuracy is:0.6590
The epoch :20,batch:19.74%,The testing accuracy is:0.6797 
----------Iteration per epoch :390--------------------
The epoch :20
The epoch cost time:1.12s
The accuracy is:0.7233
The epoch :25,batch:20.51%,The training loss is:0.1987